In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time

# Start of the timer
start_time = time.time()


col_names = ["Photo_link", "Brand", "Item_name", "Year", "Matter", "Size", "Reselling_price", "Conditions", "Modele", "Link"]
bags = pd.DataFrame(columns=col_names)

bags

,Photo_link,Brand,Item_name,Year,Matter,Size,Reselling_price,Conditions,Modele,Link


In [2]:
all_pages_bags_link = []

for number in range(1,6):
    catalogue_link = "https://www.collectorsquare.com/sacs/louis-vuitton/?filters[genders][0]=lad&filters[bag_category][0]=bag-xxxxx&filters[bag_category][1]=shoulderx&filters[bag_category][2]=shoppgbag&filters[bag_category][3]=backpabag&filters[bag_category][4]=luggage-x&filters[bag_category][5]=businessx&filters[bag_category][6]=pockthand&filters[bag_category][7]=other-bag&limit=90"+"&page="+str(number)
    
    big_page = requests.get(catalogue_link)
    big_soup = BeautifulSoup(big_page.content, 'html.parser')
    big_outermost = big_soup.find_all("li", class_="product col-sm-4 col-xs-6")
    
    bags_link = [re.search(r'<a href="(.+)" title', str(tag.find("a", {"href" :  True}))).group(1) for tag in big_outermost]
    
    for link in bags_link:
        all_pages_bags_link.append(link)

print(len(all_pages_bags_link))

420


In [3]:
for link in all_pages_bags_link:
    
    full_link = "http://www.collectorsquare.com" + str(link)
    

    # 1) We download the page using the link

    page = requests.get(full_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    outermost = soup.find_all("div", class_="product")

    # 2) We find the item_name and the price

    detailed_paraph = outermost[0].find_all("p", {"data-product-brand": True})
    item_name = detailed_paraph[0]["data-product-name"]
    reselling_price = detailed_paraph[0]["data-product-price"]

    # 3) We find the size

    detailed_paraph_2 = outermost[0].find_all("div", {"class": "infos col-md-5"})
    size = detailed_paraph_2[0].find_all("span", {"itemprop" : True})
    depth = detailed_paraph_2[0].find_all("span", {"itemprop" : "depth"})
    height = detailed_paraph_2[0].find_all("span", {"itemprop" : "height"})
    width = detailed_paraph_2[0].find_all("span", {"itemprop" : "width"})

    size_depth_or_not = re.search(r"(\d+)", str(depth))
    size_height_or_not = re.search(r"(\d+)", str(height))
    size_width_or_not = re.search(r"(\d+)", str(width))

    if size_depth_or_not and size_height_or_not and size_width_or_not:
        size_depth = re.search(r"(\d+)", str(depth)).group(1)
        size_height = re.search(r"(\d+)", str(height)).group(1)
        size_width = re.search(r"(\d+)", str(width)).group(1)
        size_total = 'x'.join([size_depth, size_height, size_width])

    else:
        size_total = "None"

    # 4) We find the conditions, the modele and the matter

    detailed_paraph_3 = detailed_paraph_2[0].find_all("div", {"class" : "col-xs-6"})

    detailed_condition_modele = detailed_paraph_3[0].find_all("div")
    conditions_or_not = re.search(r"<span>(.+)</span>", str(detailed_condition_modele[0]))
    if conditions_or_not:
        conditions = re.search(r"<span>(.+)</span>", str(detailed_condition_modele[0])).group(1)
    else:
        conditions = conditions_or_not

    modele_or_not = re.search(r"<span>(.+)</span>", str(detailed_condition_modele[3]))
    if modele_or_not:
        modele = re.search(r"<span>(.+)</span>", str(detailed_condition_modele[3])).group(1)
    else:
        modele = modele_or_not

    matter_list = [div.find('span').text for div in soup.find_all('div') if div.text.startswith('Matière')]
    if matter_list:
        matter = matter_list[0]
    else:
        matter = None

    # 5) We find the year

    detailed_year = detailed_paraph_3[0].find_all("span", {"itemprop" : True})
    year_or_not = re.search(r'<span itemprop="productionDate">(\d+)</span>', str(detailed_year))
    
    if year_or_not:
        year = int(re.search(r'<span itemprop="productionDate">(\d+)</span>', str(detailed_year)).group(1))
    else:
        year = year_or_not


    # 6) We find the photo link

    photo_incomplete = str(outermost[0]["data-product-image"])
    photo = "www.collectorsquare.com"+str(outermost[0]["data-product-image"])


    # 7) We create the new row

    new_row = {"Photo_link": photo, 
     "Brand": "VUITTON", 
     "Item_name": item_name,
     "Year" : year,
     "Matter": matter, 
     "Size": size_total, 
     "Reselling_price": int(reselling_price), 
     "Conditions": conditions, 
     "Modele": modele, 
     "Link": full_link}

    # 8) We add the new row at the end of the set

    bags.loc[len(bags)] = new_row

In [4]:
detailed_paraph_3[0]

<div class="col-xs-6">
<div>Condition : <span>Très bon état</span></div>
<div>Expert : <span>Jérôme Lalande</span></div>
<div>Genre : <span>Femme, Homme</span></div>
<div>Couleur :
                                <span itemprop="color">Marron</span></div>
<div>Matière : <span>Toile, Cuir</span></div>
</div>

In [5]:
bags.head(20)

,Photo_link,Brand,Item_name,Year,Matter,Size,Reselling_price,Conditions,Modele,Link
0,www.collectorsquare.com/images/products/346683...,VUITTON,Sac de voyage Louis Vuitton Keepall 45 en toil...,None,Toile et cuir,45x20x26,820,Très bon état,Keepall 45,http://www.collectorsquare.com/sacs/louis-vuit...
1,www.collectorsquare.com/images/products/365284...,VUITTON,Sac de voyage Louis Vuitton Keepall 60 cm en t...,2000,Toile et cuir,60x28x26,1180,Très bon état,Keepall 60,http://www.collectorsquare.com/sacs/louis-vuit...
2,www.collectorsquare.com/images/products/366710...,VUITTON,Valise rigide Louis Vuitton Pegase en toile da...,2002,Toile et cuir,36x55x16,1890,Très bon état,Pegase,http://www.collectorsquare.com/sacs/louis-vuit...
3,www.collectorsquare.com/images/products/366715...,VUITTON,Sac de voyage Louis Vuitton Keepall 55 cm en c...,1997,Cuir,55x31x25,990,Très bon état,Keepall 55,http://www.collectorsquare.com/sacs/louis-vuit...
4,www.collectorsquare.com/images/products/366703...,VUITTON,Sac de voyage Louis Vuitton Keepall 45 en toil...,2000,Toile et cuir,45x24x20,850,Très bon état,Keepall 45,http://www.collectorsquare.com/sacs/louis-vuit...
5,www.collectorsquare.com/images/products/363587...,VUITTON,Sac cabas Louis Vuitton Neverfull petit modèle...,2007,Toile et cuir,29x22x12,650,Etat d'usage,Neverfull,http://www.collectorsquare.com/sacs/louis-vuit...
6,www.collectorsquare.com/images/products/363052...,VUITTON,Sac de voyage Louis Vuitton Polochon en toile ...,1983,Toile et cuir,68x37x34,790,Très bon état,Polochon,http://www.collectorsquare.com/sacs/louis-vuit...
7,www.collectorsquare.com/images/products/364898...,VUITTON,Sac à dos Louis Vuitton Randonnée en toile mon...,None,Toile et cuir,33x43x17,650,Etat d'usage,Randonnée,http://www.collectorsquare.com/sacs/louis-vuit...
8,www.collectorsquare.com/images/products/352230...,VUITTON,Sac bandoulière Louis Vuitton Saumur grand mod...,1990,Toile et cuir,35x30x17,620,Bon état,Saumur,http://www.collectorsquare.com/sacs/louis-vuit...
9,www.collectorsquare.com/images/products/366549...,VUITTON,Sac bandoulière Louis Vuitton Saumur grand mod...,1992,Toile et cuir,39x28x25,750,Bon état,Saumur,http://www.collectorsquare.com/sacs/louis-vuit...


In [6]:
bags.drop_duplicates(inplace = True)
print(bags.shape)

print("Execution time : %s secondes ---" % (time.time() - start_time))

(420, 10)
Execution time : 170.67896485328674 secondes ---


In [7]:
bags.to_excel(excel_writer = "/Users/samitahri/Desktop/bags_dataset_4.xlsx", index = False)